In [ ]:
import numpy as np
import pandas as pd
import astropy.constants as const
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Parameters that you can change for new model
c = const.c.cgs.value #speed of light in cm/s
c_factor = 0.1 #factor to change the speed of light for the velocity of forward shock
v_fs = c_factor * c #velocity of forward shock in cm/s
dens = 1e-15 #density of agn disk

b_fs = v_fs / c
b_sf = 1 #seen on page 5
e_e = 0.3 #page 8 top right
e_b = 0.1 #page 8 top right 
xi = 1 # USED TO GET GAMMA MAX NOT SURE WHERE FROM
p = 2.5 #foud on page 8 top right
gamma_fs = 1 / np.sqrt(1 - b_fs**2)
gamma_fs_f = gamma_fs**(1 + np.sqrt(3))
gamma_sf = gamma_fs / np.sqrt(2)
gamma_sf_f = gamma_sf**(1 + np.sqrt(3))
a_gamma_sf = (gamma_sf -1)*(4*gamma_sf+3) #left of eq 24
gamma_m = 40 * (e_e/0.3)*(gamma_sf-1/0.2) #used in eq 25 from eq 16
gamma_max = 1e6 * xi**(-0.5) * (b_sf/0.5)*((a_gamma_sf/1.5)**(-0.25))*((e_b/0.1)**(-0.25))*((dens/1e-16)**(-0.25)) #used in eq 26 from eq 17
gamma_a = 60 #used in eq 27 has a messy equation at (21) pls say we dont need it
n = dens / const.m_p

def lum_sh(): #will get from emily
    if b_fs * gamma_fs <=1:
        lum_sh = 42
    else: #b_fs * gamma_fs > 1
        lum_sh = 44
    return lum_sh

def lum_syn(): #eq 15
    Yssc = (e_e / e_b)**(1/3) #page 8 top left
    Y2ndIC = (e_e / e_b)**(2/3) #page 8 top left
    f_dilution = 1 #page 6 bottom left
    if b_fs * gamma_fs <= 1:
        f_beaming = 1
    else: #b_fs * gamma_fs > 1
        f_beaming = 2 * (gamma_sf_f**2)
    lum_syn = e_e * f_dilution * f_beaming * (lum_sh() / (1 + Yssc + Y2ndIC))
    return lum_syn

def lum_tot(v):
    B_sf = 6e2*((a_gamma_sf/1.5)**0.5) * ((e_b/0.1)**0.5) * (dens/1e-16) #left of eq 24 (NOT NUMBERERED)
    v_sync = 2e9 * (gamma_sf/1.2) * (B_sf/600) #eq 24
    v_m = 3e12*((gamma_m/40)**2)*(v_sync/2e9) #eq 25
    v_max = 2e21*((gamma_max/1e6)**2)*(v_sync/2e9) #eq 26 (need to ask if i use the first or second equation)
    v_a = 7e12 * ((gamma_a/60)**2) * (v_sync/2e9) #eq 27
    
    if v_a < v_m: #eq 28
        if v_m < v < v_max:
            v_combo = (v/v_m)**(-(p-1)/2)
        if v_a < v < v_m:
            v_combo = (v/v_m)**(0.5)
        if v_sync < v < v_a:
            v_combo = ((v/v_a)**(3))*((v_a/v_m)**(0.5))
    if v_m < v_a: #eq 29
        if v_a < v < v_max:
            v_combo = (v/v_a)**(-(p-1)/2) + np.exp(1-(v/v_a)**(0.5))
        if v < v_a:
            v_combo = (v/v_a)**3

    if v_a < v_m:
        lum_tot = (lum_syn() * v_combo)/v
    if v_m < v_a:
        lum_tot = (lum_syn() * v_combo)/v


print("xi:", xi)
print("a_gamma_sf:", a_gamma_sf)
print("e_b:", e_b)
print("dens:", dens)


xi: 1
a_gamma_sf: -1.6904670343166812
e_b: 0.1
dens: 1e-15


/var/folders/pw/nc5rj5md667b29kntncq7jbw0000gn/T/ipykernel_4823/1034195725.py:25: RuntimeWarning: invalid value encountered in scalar power
  gamma_max = 1e6 * xi**(-0.5) * (b_sf/0.5)*((a_gamma_sf/1.5)**(-0.25))*((e_b/0.1)**(-0.25))*((dens/1e-16)**(-0.25)) #used in eq 26 from eq 17
